In [ ]:
!pip -q install pyecharts==2.0.5
import pandas as pd
import numpy as np
from IPython.display import display

from pyecharts.charts import Bar, Line, HeatMap
from pyecharts import options as opts
from pyecharts.globals import ThemeType
from pyecharts.commons.utils import JsCode

from pyecharts.globals import CurrentConfig
from IPython.display import display, HTML


CurrentConfig.ONLINE_HOST = "https://assets.pyecharts.org/assets/"

def show(chart):
    display(HTML(chart.render_embed()))

CSV_PATH = "/content/yrbs_2023_sadc_vars.csv"
df = pd.read_csv(CSV_PATH)


df["sleep_sufficient"] = np.where(df["sleep_8plus"].isna(), np.nan, (df["sleep_8plus"] == 1).astype(int))  # 1 means got >=8h
df["sad_yes"]          = np.where(df["sad_hopeless"].isna(), np.nan, (df["sad_hopeless"] == 1).astype(int))
df["miss_unsafe_yes"]  = np.where(df["miss_school_unsafe"].isna(), np.nan, (df["miss_school_unsafe"] == 1).astype(int))

sleep_labels = {0: "<8 hours", 1: "≥8 hours"}
grade_labels = {1: "9th", 2: "10th", 3: "11th", 4: "12th"}

def show(chart):
    display(chart.render_notebook())

def prop_yes_by_group(df_in, group_col, yes_col):
    d = df_in.dropna(subset=[group_col, yes_col]).copy()
    g = d.groupby(group_col)[yes_col].mean().sort_index()
    return g


# 1) Sleep vs Sad/Hopeless (bar)

g = prop_yes_by_group(df, "sleep_sufficient", "sad_yes")
x = [sleep_labels[i] for i in g.index.tolist()]
y = (g.values * 100).round(1).tolist()

bar1 = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.MACARONS, width="900px", height="420px"))
    .add_xaxis(x)
    .add_yaxis(
        "Sad/Hopeless (%)",
        y,
        category_gap="45%",
        label_opts=opts.LabelOpts(is_show=True, position="top", formatter="{c}%"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sleep vs Sad/Hopeless", subtitle="YRBS 2023 SADC (National)"),
        yaxis_opts=opts.AxisOpts(name="Percent", axislabel_opts=opts.LabelOpts(formatter="{value}%")),
        xaxis_opts=opts.AxisOpts(name="Sleep (school nights)"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
)
show(bar1)


# 2) Sleep vs Missed School Unsafe (bar)

g = prop_yes_by_group(df, "sleep_sufficient", "miss_unsafe_yes")
x = [sleep_labels[i] for i in g.index.tolist()]
y = (g.values * 100).round(2).tolist()

bar2 = (
    Bar(init_opts=opts.InitOpts(theme=ThemeType.CHALK, width="900px", height="420px"))
    .add_xaxis(x)
    .add_yaxis(
        "Missed school due to feeling unsafe (%)",
        y,
        category_gap="45%",
        label_opts=opts.LabelOpts(is_show=True, position="top", formatter="{c}%"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sleep vs Attendance Disruption", subtitle="Missed school because felt unsafe"),
        yaxis_opts=opts.AxisOpts(name="Percent", axislabel_opts=opts.LabelOpts(formatter="{value}%")),
        xaxis_opts=opts.AxisOpts(name="Sleep (school nights)"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
)
show(bar2)


# 3) Sleep trend by grade (line)

sub = df.dropna(subset=["grade", "sleep_sufficient"]).copy()
sleep_by_grade = sub.groupby("grade")["sleep_sufficient"].mean().sort_index()

x = [grade_labels.get(int(i), str(int(i))) for i in sleep_by_grade.index.tolist()]
y = (sleep_by_grade.values * 100).round(1).tolist()

line1 = (
    Line(init_opts=opts.InitOpts(theme=ThemeType.ROMA, width="950px", height="420px"))
    .add_xaxis(x)
    .add_yaxis(
        series_name="Sleep ≥8 hours (%)",
        y_axis=y,
        is_smooth=True,
        symbol="circle",
        label_opts=opts.LabelOpts(is_show=True, formatter="{c}%"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Sleep Sufficiency Trend by Grade", subtitle="Share reporting ≥8 hours on school nights"),
        yaxis_opts=opts.AxisOpts(name="Percent", axislabel_opts=opts.LabelOpts(formatter="{value}%")),
        xaxis_opts=opts.AxisOpts(name="Grade level"),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
    )
)
show(line1)


# 4) Heatmap: Sleep × Sadness → Missed School Unsafe

sub = df.dropna(subset=["sleep_sufficient", "sad_yes", "miss_unsafe_yes"]).copy()

pivot = sub.pivot_table(
    index="sad_yes",
    columns="sleep_sufficient",
    values="miss_unsafe_yes",
    aggfunc="mean"
).reindex(index=[0, 1], columns=[0, 1])

x_labels = ["<8 hours", "≥8 hours"]
y_labels = ["Not sad/hopeless", "Sad/hopeless"]

data = []
for yi, sad_val in enumerate([0, 1]):
    for xi, sleep_val in enumerate([0, 1]):
        v = pivot.loc[sad_val, sleep_val]
        data.append([xi, yi, float(v * 100)])  # percent

vals = [d[2] for d in data if not np.isnan(d[2])]
vmin, vmax = (min(vals), max(vals)) if vals else (0, 1)

hm = (
    HeatMap(init_opts=opts.InitOpts(theme=ThemeType.VINTAGE, width="900px", height="450px"))
    .add_xaxis(x_labels)
    .add_yaxis(
        "Missed school due to unsafe (%)",
        y_labels,
        data,
        label_opts=opts.LabelOpts(
            is_show=True,
            formatter=JsCode("function(p){return p.data[2].toFixed(1) + '%';}")
        ),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="Sleep × Stress → Attendance Disruption",
            subtitle="Heatmap shows % who missed school due to feeling unsafe"
        ),
        visualmap_opts=opts.VisualMapOpts(
            min_=vmin,
            max_=vmax,
            is_calculable=True,
            orient="horizontal",
            pos_left="center",
        ),
        toolbox_opts=opts.ToolboxOpts(is_show=True),
        xaxis_opts=opts.AxisOpts(name="Sleep (school nights)"),
        yaxis_opts=opts.AxisOpts(name="Sad/hopeless status"),
    )
)
show(hm)

from IPython.display import IFrame

def show_iframe(chart, name="chart.html", w=950, h=520):
    chart.render(name)
    return IFrame(src=name, width=w, height=h)


show(bar1)
show(bar2)
show(line1)
show(hm)
